In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv(r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\Resourses_Prj_3\.env')
def fetch_articles(ticker, start_date='2019-01-01', end_date= None, max_articles=1000): # change parameters to (ticker, start_date, end_date=None, max_articles=1000)
    api_key = os.getenv('NYT_API_KEY')
    if not api_key:
        raise Exception("API key not found. Please ensure your .env file is configured correctly and the environment variable is loaded.")
    else:
        print("API Key Loaded Successfully:", api_key)
    api_key = os.getenv('NYT_API_KEY')
    if not api_key:
        raise Exception("API key not found. Please ensure your .env file is configured correctly.")
    
    all_articles = []
    page = 0
    while True:
        url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={ticker}&page={page}&api-key={api_key}"
        print("Requesting URL:", url)  # Debug: print the URL being requested
        response = requests.get(url)
        
        if response.status_code != 200:
            print("Failed to fetch articles:", response.status_code, response.text)  # Error handling
            break
        
        data = response.json()
        print("API Response:", data)  # Debug: print raw API response

        articles = data.get('response', {}).get('docs', [])
        
        if not articles:
            print("No more articles found.")  # Debug: inform when no more articles are found
            break
        
        for article in articles:
            snippet = article.get('snippet', '')
            pub_date = article.get('pub_date', '')
            all_articles.append({'date': pub_date, 'snippet': snippet})
        
        page += 1
        if page > 1:  # Limit to first 20 results for demo purposes
            break
    
    return pd.DataFrame(all_articles)

# Example usage
ticker = 'IBM'
articles_df = fetch_articles(ticker)
print(articles_df.head())
print(articles_df.info())

API Key Loaded Successfully: uhnzXeb2hYPHJy6hIsBU0Ie8kSUkncRY
Requesting URL: https://api.nytimes.com/svc/search/v2/articlesearch.json?q=IBM&page=0&api-key=uhnzXeb2hYPHJy6hIsBU0Ie8kSUkncRY
API Response: {'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'abstract': 'Technical failures blocked thousands of students who tried to log on to school during a snowstorm. At a City Council hearing on Wednesday, an IBM official said it wasn’t the company’s fault.', 'web_url': 'https://www.nytimes.com/2024/03/06/nyregion/remote-class-technical-problems-nyc-schools-ibm.html', 'snippet': 'Technical failures blocked thousands of students who tried to log on to school during a snowstorm. At a City Council hearing on Wednesday, an IBM official said it wasn’t the company’s fault.', 'lead_paragraph': 'When New York City held virtual classes during a winter storm last month, the system ran into major tech problems. But officials unf

In [3]:
#!pip install transformers torch

In [4]:

print(articles_df.head())
print(articles_df.info())

                       date                                            snippet
0  2024-03-06T22:31:20+0000  Technical failures blocked thousands of studen...
1  2024-02-13T16:49:02+0000  The chancellor said the “school system is more...
2  2023-12-21T20:21:55+0000  Suggested reading from critics and editors at ...
3  2023-12-11T14:33:36+0000  The move is aimed at drawing $9 billion in cor...
4  2023-11-24T10:02:14+0000  As the downsides of 401(k)-style plans become ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     10 non-null     object
 1   snippet  10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes
None


In [5]:
from transformers import pipeline

# Load a pre-trained sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")
# The function for Error handling and API call limit handling
def safe_analyze_sentiments(batch):
    try:
        return sentiment_analyzer(batch)
    except Exception as e:
        print("Error analyzing sentiments:", e)
        # Return neutral sentiment and 0 confidence if an error occurs
        return [{'label': 'NEUTRAL', 'score': 0.0} for _ in batch]
# The function for sentiment analysis
def analyze_sentiments(articles_df, batch_size=10):
    # list to store results
    sentiments=[]
    confidence=[]
    # loop through snippets in batches
    for i in range(0, len(articles_df), batch_size):
        batch = articles_df['snippet'].iloc[i:i+batch_size].tolist()
        results = sentiment_analyzer(batch)
        for result in results:
            sentiments.append(result['label'])
            confidence.append(result['score'])
    # Check if 'snippet' column exists
    if 'snippet' not in articles_df.columns:
        raise ValueError("Snippet column missing from articles data. Check the data extraction logic.")
    # Applying the sentiment analysis model to each snippet
    articles_df['sentiment'] = articles_df['snippet'].apply(lambda x: sentiment_analyzer(x)[0])
    return articles_df

# enter in stock ticker and run the function
articles_df = fetch_articles('IBM')  #exapmple of IBM, which is the stock ticker for IBM
sentiment_df = analyze_sentiments(articles_df)
print(sentiment_df[['date', 'snippet', 'sentiment']])


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


API Key Loaded Successfully: uhnzXeb2hYPHJy6hIsBU0Ie8kSUkncRY
Requesting URL: https://api.nytimes.com/svc/search/v2/articlesearch.json?q=IBM&page=0&api-key=uhnzXeb2hYPHJy6hIsBU0Ie8kSUkncRY
API Response: {'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'abstract': 'Technical failures blocked thousands of students who tried to log on to school during a snowstorm. At a City Council hearing on Wednesday, an IBM official said it wasn’t the company’s fault.', 'web_url': 'https://www.nytimes.com/2024/03/06/nyregion/remote-class-technical-problems-nyc-schools-ibm.html', 'snippet': 'Technical failures blocked thousands of students who tried to log on to school during a snowstorm. At a City Council hearing on Wednesday, an IBM official said it wasn’t the company’s fault.', 'lead_paragraph': 'When New York City held virtual classes during a winter storm last month, the system ran into major tech problems. But officials unf

In [6]:
import pandas as pd

# Convert sentiment date to datetime
sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.date
# rename the 'date' column as ds
sentiment_df.rename(columns={'date': 'ds'}, inplace=True)

# save sentiment data to a csv file
def save_sentiment_data(sentiment_df, ticker):
    filepath = r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\sentiment_IBM.csv'
    sentiment_df.to_csv(filepath, index=False)
    print(f"Sentiment data saved to .csv")
    
# execute the function
save_sentiment_data(sentiment_df, 'IBM')

Sentiment data saved to .csv


In [7]:
# merge the sentiment data with the prophet data
forecast_results= pd.read_csv(r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\forecast_results_IBM.csv')
sentiment_results= pd.read_csv(r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\sentiment_IBM.csv')
print(forecast_results.head(2))
print(sentiment_results.head(2))

           ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0  2019-01-02  118.734058  114.227785  130.136490   118.734058   118.734058   
1  2019-01-03  118.819730  113.777178  129.548908   118.819730   118.819730   

   additive_terms  additive_terms_lower  additive_terms_upper    weekly  \
0        2.895686              2.895686              2.895686  1.910086   
1        2.835698              2.835698              2.835698  1.820278   

   weekly_lower  weekly_upper   yearly  yearly_lower  yearly_upper  \
0      1.910086      1.910086  0.98560       0.98560       0.98560   
1      1.820278      1.820278  1.01542       1.01542       1.01542   

   multiplicative_terms  multiplicative_terms_lower  \
0                   0.0                         0.0   
1                   0.0                         0.0   

   multiplicative_terms_upper        yhat  
0                         0.0  121.629744  
1                         0.0  121.655428  
           ds              

In [8]:
#check date format in both .csv files
forecast_results['ds'] = pd.to_datetime(forecast_results['ds'])
sentiment_results['ds'] = pd.to_datetime(sentiment_results['ds'])

# Merge  the Sentiment and Forecast .csv into one file
integrated_IBM_data= pd.merge(forecast_results, sentiment_results, on='ds', how='outer')

# forward fill the sentiment data. This is a common practice in financial analysis to fill missing values.
# the latest sentiment value mainatined until a new value is available
sentiment_results['sentiment'].fillna(method= 'ffill', inplace=True)
integrated_IBM_data['sentiment'].fillna(method= 'ffill', inplace=True)

# Save the integrated data to a .csv file
integrated_IBM_data.to_csv(r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\integrated_IBM_data.csv', index=False)
integrated_IBM_data.head(2)




C:\Users\mikea\AppData\Local\Temp\ipykernel_501556\3126879971.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sentiment_results['sentiment'].fillna(method= 'ffill', inplace=True)
C:\Users\mikea\AppData\Local\Temp\ipykernel_501556\3126879971.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sentiment_results['sentiment'].fillna(method= 'ffill', inplace=True)
C:\Users\mikea\AppData\Local\Temp\ipykernel_501556\3126879971.py:11: Future

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,...,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,snippet,sentiment
0,1985-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'label': 'POSITIVE', 'score': 0.7481210231781..."
1,1985-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'label': 'POSITIVE', 'score': 0.7481210231781..."


In [9]:
#!pip install fpdf


In [10]:
from fpdf import FPDF

# set path for pdf and csv files
pdf_path = r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\integrated_IBM_data.pdf'
csv_path = r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\integrated_IBM_data.csv'

def Convert_csv_to_pdf(pdf_path, csv_path):
    Integrated_IBM_pdf = FPDF()
    Integrated_IBM_pdf.add_page()
    Integrated_IBM_pdf.set_font("Arial", size=2)
    with open(csv_path, "r", encoding='utf-8') as f:
        for line in f:
            # Replace right single quotation marks and other problematic characters
            line = line.replace(u'\u2019', '\'').replace(u'\u201c', '\"').replace(u'\u201d', '\"')
            # Add line to PDF, encoding explicitly as ISO-8859-1
            Integrated_IBM_pdf.cell(200, 10, txt=line.encode('latin-1', 'replace').decode('latin-1'), ln=True, align='C')
    Integrated_IBM_pdf.output(pdf_path)
    print("PDF Created")

pdf_path = r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\integrated_IBM_data.pdf'
csv_path = r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\integrated_IBM_data.csv'
Convert_csv_to_pdf(pdf_path, csv_path)


PDF Created
